In [ ]:
import glob
import os

import geopandas as gpd
from tqdm.notebook import tqdm
from s1cs2lib import s1_nc_geolocation, proj_nps, proj_pc, Polygon
from nansat import Nansat

In [ ]:
def get_metadata_nc(ifile):
    self = s1_nc_geolocation(ifile)
    self.read_geolocation()
    self.prepare_interpolators()
    polygon = self.get_border_polygon()
    start_date = self.get_datetime()
    return polygon, start_date

def get_metadata_safe_zip(ifile):
    n = Nansat(ifile, fast=True)
    border_lon, border_lat = n.get_border()
    border_x, border_y, _ = proj_nps.transform_points(proj_pc, border_lon, border_lat).T
    return Polygon(zip(border_x, border_y)), n.time_coverage_start

In [ ]:
odir = f'/data1/{os.getenv("USER")}/foccus'

# ALL APASIP DATA for Jan 2020
#idirs = [
#    '/Data/sat/downloads/sentinel1/apasip/202001',
#]
#ext = '*.nc'
#metadata_function = get_metadata_nc
#ofile = 'apasip_202001.feather'

# Svalbard data for 2019
#idirs = [
#    '/Data/sat/downloads/sentinel1/2019/svalbard',
#]
#ext = '*.zip'
#metadata_function = get_metadata_safe_zip
#ofile = 'svalbard_2019.feather'

# Greenland data for 2019
#idirs = [
#    '/Data/sat/downloads/sentinel1/2019/greenland',
#]
#ext = '*.zip'
#metadata_function = get_metadata_safe_zip
#ofile = 'greenland_2019.feather'

# Greenland data for Feb, March 2020
#idirs = [
#    '/Data/sat/downloads/sentinel1/2020/greenland',
#]
#ext = '*.zip'
#metadata_function = get_metadata_safe_zip
#ofile = 'greenland_2020.feather'

# Svalbard data for Feb, March 2020
idirs = [
    '/Data/sat/downloads/sentinel1/2020/svalbard',
]
ext = '*.zip'
metadata_function = get_metadata_safe_zip
ofile = 'svalbard_2020.feather'

In [ ]:
ifiles = []
for idir in idirs:
    ifiles.extend(glob.glob(f'{idir}/{ext}'))
len(ifiles)

In [ ]:
orbits = [int(os.path.basename(ifile).split('_')[6]) for ifile in ifiles]
dates = []
polygons = []
for ifile in tqdm(ifiles):
    polygon, start_date = metadata_function(ifile)
    polygons.append(polygon)
    dates.append(start_date)


In [ ]:
gdf = gpd.GeoDataFrame(dict(name=ifiles, orbit=orbits, geometry=polygons, date=dates))
gdf.to_feather(f'{odir}/{ofile}')